## Simple prologterms example

Walkthrough of basic commands

In [2]:
from prologterms import TermGenerator, PrologRenderer, Var

# term generators allow for convient construction of
# compound terms
P = TermGenerator()

# prolog variables; must start with uppercase
X = Var('X')

# create a compound term object, representing
# a non-ground prolog goal
term = P.member(X, [1, 2, 3])

In [4]:
from prologterms import PrologRenderer
r = PrologRenderer()
print(r.render(term))

member(X, [1, 2, 3])


### escaping characters

this module will take care of escaping characters to conform to
prolog syntax


In [5]:
term = P.member(X, [1, 'b', "foo\nbar", "'hi'"])
print(r.render(term))

member(X, [1, b, 'foo\nbar', '\'hi\''])


### Rules

The "<=" operator in python is overloaded to mean the same as prolog
":-". This means a rule object can be created as follows:

In [7]:
X = Var('X')
Y = Var('Y')
Z = Var('Z')

rule = P.ancestor(X,Y) <= (P.parent(X,Z), P.ancestor(Z,Y))
print(r.render(rule))

ancestor(X, Y) :-
    parent(X, Z), ancestor(Z, Y)


### Adding comments



In [9]:
rule = (P.ancestor(X,Y) <= (P.parent(X,Z), P.ancestor(Z,Y))) % 'the ancestor relation propagates over parent/2'
print(r.render(rule))

% the ancestor relation propagates over parent/2
ancestor(X, Y) :-
    parent(X, Z), ancestor(Z, Y)


### Programs


In [32]:
from prologterms import TermGenerator, PrologRenderer, Program, Var

P = TermGenerator()
X = Var('X')
Y = Var('Y')
Z = Var('Z')
R = PrologRenderer()

prog = Program(
    P.ancestor(X,Y) <= (P.parent(X,Z), P.ancestor(Z,Y)),
    P.ancestor(X,Y) <= P.parent(X,Y),
    P.parent('a','b'),
    P.parent('b','c'),
    P.parent('c','d')
)

print(R.render(prog))

ancestor(X, Y) :-
    parent(X, Z), ancestor(Z, Y).
ancestor(X, Y) :-
    parent(X, Y).
parent(a, b).
parent(b, c).
parent(c, d).



## Interfacing with prolog systems

we show two ways of doing this

### Interfacing with SWI-Prolog via commandline

This requires `swipl` on command line to run

In [29]:
# write program
f = open('anc.pl', 'w')
f.write(R.render(prog))
f.close()

# query
q = P.ancestor(X,Y)

# make a wrapper goal
goal = (P.forall(q, P.writeln(P.anc(X,Y))),'halt')

# call
cmd = "swipl -l anc.pl -g '{}'".format(R.render(goal))
print(cmd)

swipl -l anc.pl -g '(forall(ancestor(X, Y), writeln(anc(X, Y))), halt)'


In [30]:
import subprocess
from subprocess import run
result = run(cmd, shell=True, stdout=subprocess.PIPE)
print("{}".format(result.stdout.decode('ascii')))

anc(a,d)
anc(a,c)
anc(b,d)
anc(a,b)
anc(b,c)
anc(c,d)



### Interfacing via pengines

[pengines](http://pengines.swi-prolog.org/docs/index.html) (Prolog Engines) is a means of communicating with a prolog server from any language. We will use the python pengines API here (TODO: release to PyPi).

This example requires a pengines server to be running on port 4242. This is fairly easy to do using the pengines framework, and even easier if you have docker, in which case you can run a container like this:

```
docker run -p 4242:9083 -e PORT=9083 --name  cmungall/sparqlprog
```

Once this is done, the following program should run.

In [36]:
from pengines.Builder import PengineBuilder
from pengines.Pengine import Pengine

factory = PengineBuilder(urlserver="http://localhost:4242",
                         srctext=R.render(prog),
                         ask=R.render(q))
pengine = Pengine(builder=factory, debug=False)
while pengine.currentQuery.hasMore:
    pengine.doNext(pengine.currentQuery)
for p in pengine.currentQuery.availProofs:
    print('{} <- {}'.format(p[X.name], p[Y.name]))

Builder._getActualURL log.
New value is : http://localhost:4242/pengine/create
a <- d
a <- c
b <- d
a <- b
b <- c
c <- d
